In [18]:
%reload_ext autoreload
%autoreload 2

from pprint import pprint
import csv
from copy import deepcopy
import json
import pathlib
from pathlib import Path
from openai import OpenAI
from dotenv import load_dotenv
import src.annotation_utils as a_utils
import src.llm_utils as llm_utils
import src.message_utils as m_utils
import os

load_dotenv()

BRAT_DATA_PATH = os.environ['BRAT_DATA_PATH']
DATA_DEF_FILE = os.environ['DATA_DEF_FILE']


## Load evaluation dataset

### Load from a previous fine-tune job

In [ ]:
job_desc_dir = None

training_set, validation_set, test_set, fine_tuned_model_id = llm_utils.load_eval_info(job_desc_dir)

len(training_set), len(validation_set), len(test_set)

### Or, Load a fresh dataset

In [2]:
## For (segment, data_span)
data_entities = a_utils.load_data_entities_of_segments(BRAT_DATA_PATH, DATA_DEF_FILE)

data_entities = [segment for segment in data_entities if segment['entities']]

# For data span
# all_data = m_utils.as_training_data_for_data_span_of_segment(data_entities)
# For data classification
# all_data = m_utils.as_training_data_for_data_classification_of_segment(data_entities)
# For data classification (gradual, level 0)
all_data = m_utils.as_training_data_for_data_classification_of_segment_gradual(data_entities)

## For (segment, sentence, data_span)
# data_entities = a_utils.load_data_entities_of_sentences(BRAT_DATA_PATH, DATA_DEF_FILE)
# For data span of sentence
# all_data = m_utils.as_training_data_for_data_span_of_sentence(data_entities)

test_set = all_data

len(test_set)
test_set

[{'messages': [{'role': 'system',
    'content': 'You are an annotation expert, and your task is to map phrases concerning user private data to one of the high-level categories below.\nYou will be given a list of categories/terms, belong to the same level of a category hierarchy. You will also be given definitions of each of these categories. Please rely on these definitions.\nThe category list is between `<category>...</category>`, and each row describes one category: the first element the first element is the category name, and the second element is the category description; they are separated using a comma (`,`); everything after the first comma is part of the description of the corresponding category.\nYou always have to choose the category that matches the given phrase the closest and most precise.\nHere is the categories:\n\n<category>\n(\'Internal, Information about internal characteristics that cannot be seen or observed\\nExternal, Information about external characteristics th

## Run model evaluation

In [3]:
model_id = fine_tuned_model_id if 'fine_tuned_model_id' in locals() else 'gpt-4o-mini-2024-07-18'
# model_id = '4.0Ultra'
model_id

'gpt-4o-mini-2024-07-18'

In [7]:
messages_list = [data['messages'][:-1] for data in test_set]
correct_outputs = [data['messages'][-1]['content'] for data in test_set]
dir_name, obj_model_outputs = llm_utils.query_llm(model_id, messages_list, correct_outputs=correct_outputs,
                                                  desc='data_category-seg_entity-gradual-level_0')

dir_name, len(obj_model_outputs)

Constructing batch data: 100%|██████████| 45/45 [00:00<00:00, 584345.76it/s]


('eval-2024-09-16-23-07-21-gpt-4o-mini-2024-07-18', 0)

In [19]:
llm_utils.retrieve_batch_query_result()
llm_utils.combine_batch_query_result()